In [1]:
from teemi.design.fetch_sequences import retrieve_sequences_from_ncbi, read_fasta_files

OneHot encoding

In [5]:
from sklearn.preprocessing import OneHotEncoder
import numpy as np

def one_hot_encode(sequence):
    # Define the list of amino acids
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']
    
    # Create a dictionary to map each amino acid to an integer
    aa_to_int = dict((aa, i) for i, aa in enumerate(amino_acids))
    
    # Convert the sequence to a list of integers
    seq_int = [aa_to_int[aa] for aa in sequence]
    
    # Reshape the integer sequence to a 2D array
    seq_int = np.array(seq_int).reshape(-1, 1)
    
    # Initialize the OneHotEncoder
    enc = OneHotEncoder(categories='auto', sparse=False, dtype=np.int8)
    
    # Fit and transform the integer sequence using the OneHotEncoder
    one_hot = enc.fit_transform(seq_int)
    
    return one_hot

In [10]:
sequence = 'MACDEFGHIKLMNPQRSTVWY----------'
one_hot = one_hot_encode(sequence)
print(len(one_hot))


31


In [11]:
def one_hot_decode(one_hot):
    # Define the list of amino acids
    amino_acids = ['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', '-']
    
    # Get the index of the maximum value in each row of the one-hot encoded sequence
    max_index = np.argmax(one_hot, axis=1)
    
    # Convert the index sequence to a list of amino acids
    sequence = [amino_acids[i] for i in max_index]
    
    # Join the list of amino acids into a string
    sequence = ''.join(sequence)
    
    return sequence

amino_decode = one_hot_decode(one_hot)
amino_decode

'MACDEFGHIKLMNPQRSTVWY----------'

In [2]:
acc_numbers = ['Q8VWZ7',
 'AGX93053',
 'AGX93055',
 'AGX93051',
 'BAP90522',
 'D1MI46',
 'XP_022858342',
 'AES93118']

In [3]:
chosen_g8h_ncbi_fasta = retrieve_sequences_from_ncbi(acc_numbers,'../data/dummy_protein_seq/dummy_seqs.fasta')


In [4]:
fasta_seqs = read_fasta_files('../data/dummy_protein_seq/dummy_seqs.fasta')

In [5]:
fasta_seqs

[SeqRecord(seq=Seq('MDYLTIILTLLFALTLYEAFSYLSRRTKNLPPGPSPLPFIGSLHLLGDQPHKSL...STL'), id='sp|Q8VWZ7.1|C76B6_CATRO', name='sp|Q8VWZ7.1|C76B6_CATRO', description='sp|Q8VWZ7.1|C76B6_CATRO RecName: Full=Geraniol 8-hydroxylase; AltName: Full=Cytochrome P450 76B6; AltName: Full=Geraniol 10-hydroxylase; Short=CrG10H', dbxrefs=[]),
 SeqRecord(seq=Seq('MDYLTITLGLLFALTFYQGLSYLSRRSKKLPPGPAPLPIIGNLHMLGDQPHKSL...IPF'), id='AGX93053.1', name='AGX93053.1', description='AGX93053.1 geraniol 10-hydroxylase-like protein [Rauvolfia serpentina]', dbxrefs=[]),
 SeqRecord(seq=Seq('MDYLTIALGLLFALTFYQGLTYLSRRSQKLPPGPLPLPIIGNLHLLGDHPHKSL...SPL'), id='AGX93055.1', name='AGX93055.1', description='AGX93055.1 geraniol 10-hydroxylase-like protein [Vinca minor]', dbxrefs=[]),
 SeqRecord(seq=Seq('MDYLTIVLGLLFALTLYQGLSSLSRKAKKLPPGPTPLPIIGNLHLLGDQPHKSL...ISL'), id='AGX93051.1', name='AGX93051.1', description='AGX93051.1 geraniol 10-hydroxylase-like protein [Cinchona calisaya]', dbxrefs=[]),
 SeqRecord(seq=Seq('MRILDSEVSKI

In [6]:
from dnachisel.biotools import reverse_translate

In [7]:
#add  * to indicate stop codon - and reverse traslate it
for seq in fasta_seqs: 
    seq.seq = reverse_translate(seq.seq)

In [8]:
fasta_seqs

[SeqRecord(seq='ATGGATTATTTAACTATTATTTTAACTTTATTATTTGCTTTAACTTTATATGAAGCTTTTTCTTATTTATCTCGTCGTACTAAAAATTTACCTCCTGGTCCTTCTCCTTTACCTTTTATTGGTTCTTTACATTTATTAGGTGATCAACCTCATAAATCTTTAGCTAAATTATCTAAAAAACATGGTCCTATTATGTCTTTAAAATTAGGTCAAATTACTACTATTGTTATTTCTTCTTCTACTATGGCTAAAGAAGTTTTACAAAAACAAGATTTAGCTTTTTCTTCTCGTTCTGTTCCTAATGCTTTACATGCTCATAATCAATTTAAATTTTCTGTTGTTTGGTTACCTGTTGCTTCTCGTTGGCGTTCTTTACGTAAAGTTTTAAATTCTAATATTTTTTCTGGTAATCGTTTAGATGCTAATCAACATTTACGTACTCGTAAAGTTCAAGAATTAATTGCTTATTGTCGTAAAAATTCTCAATCTGGTGAAGCTGTTGATGTTGGTCGTGCTGCTTTTCGTACTTCTTTAAATTTATTATCTAATTTAATTTTTTCTAAAGATTTAACTGATCCTTATTCTGATTCTGCTAAAGAATTTAAAGATTTAGTTTGGAATATTATGGTTGAAGCTGGTAAACCTAATTTAGTTGATTTTTTTCCTTTATTAGAAAAAGTTGATCCTCAAGGTATTCGTCATCGTATGACTATTCATTTTGGTGAAGTTTTAAAATTATTTGGTGGTTTAGTTAATGAACGTTTAGAACAACGTCGTTCTAAAGGTGAAAAAAATGATGTTTTAGATGTTTTATTAACTACTTCTCAAGAATCTCCTGAAGAAATTGATCGTACTCATATTGAACGTATGTGTTTAGATTTATTTGTTGCTGGTACTGATACTACTTCTTCTACTTTAGAATGGGCTATGTCTGAAATGTTAAAAAATCCTGATAAAATGAAAAAAACTCAAGATGAATTAGCT

In [65]:
from dnachisel import DnaOptimizationProblem, EnforceGCContent, CodonOptimize
from typing import List
from Bio.SeqRecord import SeqRecord

def codon_optimize_with_dnachisel(
    sequences: List[SeqRecord],
    lower_GC: float = 0.1,
    upper_GC: float = 0.9,
    species: str = None,
    codon_usage_table = None,
    window: int = 100,
    method : str = 'use_best_codon'
) -> List[SeqRecord]:
    
    """Codon-optimize sequences with_dnachisel.

    Parameters
    ----------
    sequences : list
        list of Bio.SeqRecord objects
    lower_GC : float
        the lowest GC content in the region of 50 bp
    upper_GC : float
        the highest GC content in the region of 50 bp
    species : str
        name of the species for which to optimize the sequence.
        examples: 'e_coli, s_cerevisiae, h_sapiens, c_elegans, b_subtilis, d_melanogaster
        check python_codon_tables for more info.
    codon_usage_table:
        a codon table following the structure of:
        {'*': {'TAA': 0.0, 'TAG': 0.0, 'TGA': 1.0},...
    method : str 
        Either ‘use_best_codon’, ‘match_codon_usage’, or ‘harmonize_rca’. Default is ‘use_best_codon’.
        

    Returns
    -------
    list of codon optimized sequences for yeast
    """
    
    
    if not species and not codon_usage_table:
        raise ValueError("At least one of `species` and `codon_usage_table` must be specified.")
    
    codon_optimized_seqs = []
    

    # DEFINE THE OPTIMIZATION PROBLEM
    for seq in sequences:
        if species: 
            problem = DnaOptimizationProblem(
                sequence=str(seq.seq),
                objectives=[CodonOptimize(species=species, method=method)],
            )

            # SOLVE THE CONSTRAINTS, OPTIMIZE WITH RESPECT TO THE OBJECTIVE
            #problem.resolve_constraints()
            problem.optimize()
            
            
            print(problem.objectives_text_summary())

        if codon_usage_table: 
            problem = DnaOptimizationProblem(
                                        sequence=str(seq.seq),
                                        objectives=[CodonOptimize(codon_usage_table=codon_usage_table, method=method)])

            # SOLVE THE CONSTRAINTS, OPTIMIZE WITH RESPECT TO THE OBJECTIVE
            #problem.resolve_constraints()
            problem.optimize()

            print(problem.objectives_text_summary())

        # GET THE FINAL SEQUENCE AS ANNOTATED BIOPYTHON RECORDS)
        final_record = problem.to_record(with_sequence_edits=True)
        final_record.id = seq.id
        final_record.name = seq.name
        final_record.description = seq.description

        codon_optimized_seqs.append(final_record)

    return codon_optimized_seqs

In [66]:
A_oryzae_rib40 = {'*': {'TAA': 0.0, 'TAG': 0.0, 'TGA': 1.0},
 'A': {'GCA': 0.07, 'GCC': 0.4, 'GCG': 0.33, 'GCT': 0.2},
 'C': {'TGC': 1.0, 'TGT': 0.0},
 'D': {'GAC': 0.87, 'GAT': 0.13},
 'E': {'GAA': 0.14, 'GAG': 0.86},
 'F': {'TTC': 1.0, 'TTT': 0.0},
 'G': {'GGA': 0.11, 'GGC': 0.42, 'GGG': 0.32, 'GGT': 0.16},
 'H': {'CAC': 1.0, 'CAT': 0.0},
 'I': {'ATA': 0.0, 'ATC': 0.93, 'ATT': 0.07},
 'K': {'AAA': 0.0, 'AAG': 1.0},
 'L': {'CTA': 0.0,
  'CTC': 0.42,
  'CTG': 0.54,
  'CTT': 0.04,
  'TTA': 0.0,
  'TTG': 0.0},
 'M': {'ATG': 1.0},
 'N': {'AAC': 0.83, 'AAT': 0.17},
 'P': {'CCA': 0.06, 'CCC': 0.18, 'CCG': 0.65, 'CCT': 0.12},
 'Q': {'CAA': 0.2, 'CAG': 0.8},
 'R': {'AGA': 0.0,
  'AGG': 0.09,
  'CGA': 0.12,
  'CGC': 0.35,
  'CGG': 0.35,
  'CGT': 0.09},
 'S': {'AGC': 0.27,
  'AGT': 0.0,
  'TCA': 0.0,
  'TCC': 0.09,
  'TCG': 0.45,
  'TCT': 0.18},
 'T': {'ACA': 0.08, 'ACC': 0.5, 'ACG': 0.35, 'ACT': 0.08},
 'V': {'GTA': 0.12, 'GTC': 0.32, 'GTG': 0.56, 'GTT': 0.0},
 'W': {'TGG': 1.0},
 'Y': {'TAC': 0.89, 'TAT': 0.11}}

In [67]:
codon_optimized_seqs = codon_optimize_with_dnachisel(fasta_seqs,codon_usage_table = A_oryzae_rib40 )
#codon_optimized_seqs = codon_optimize_with_dnachisel(fasta_seqs,species ='s_cerevisiae' )


location:   0%|                                | 0/472 [00:00<?, ?it/s, now=3-6]


===> TOTAL OBJECTIVES SCORE:         0
✔        0 ┍ MaximizeCAI[0-1479]((custom table)) 
           │ Codon opt. on window 0-1479 scored -0.00E+00




location:   0%|                                | 0/474 [00:00<?, ?it/s, now=3-6]


===> TOTAL OBJECTIVES SCORE:         0
✔        0 ┍ MaximizeCAI[0-1479]((custom table)) 
           │ Codon opt. on window 0-1479 scored -0.00E+00




location:   0%|                                | 0/475 [00:00<?, ?it/s, now=3-6]


...

===> TOTAL OBJECTIVES SCORE:         0
✔        0 ┍ MaximizeCAI[0-1485]((custom table)) 
           │ Codon opt. on window 0-1485 scored -0.00E+00




location:   0%|                                | 0/473 [00:00<?, ?it/s, now=3-6]


===> TOTAL OBJECTIVES SCORE:         0
✔        0 ┍ MaximizeCAI[0-1482]((custom table)) 
           │ Codon opt. on window 0-1482 scored -0.00E+00




location:   0%|                                | 0/476 [00:00<?, ?it/s, now=3-6]
                                                                                

===> TOTAL OBJECTIVES SCORE:         0
✔        0 ┍ MaximizeCAI[0-1503]((custom table)) 
           │ Codon opt. on window 0-1503 scored -0.00E+00




In [68]:
codon_optimized_seqs

[SeqRecord(seq=Seq('ATGGAGTACTTCACCATGATGTTCACCTTCTTCTTCGCCTTCACCTTCTACGAC...TTC'), id='sp|Q8VWZ7.1|C76B6_CATRO', name='sp|Q8VWZ7.1|C76B6_CATRO', description='sp|Q8VWZ7.1|C76B6_CATRO RecName: Full=Geraniol 8-hydroxylase; AltName: Full=Cytochrome P450 76B6; AltName: Full=Geraniol 10-hydroxylase; Short=CrG10H', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGAGTACTTCACCATGACCTTCGGCTTCTTCTTCGCCTTCACCTTCTACCAC...TTC'), id='AGX93053.1', name='AGX93053.1', description='AGX93053.1 geraniol 10-hydroxylase-like protein [Rauvolfia serpentina]', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGAGTACTTCACCATGGCCTTCGGCTTCTTCTTCGCCTTCACCTTCTACCAC...TTC'), id='AGX93055.1', name='AGX93055.1', description='AGX93055.1 geraniol 10-hydroxylase-like protein [Vinca minor]', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGGAGTACTTCACCATGGTGTTCGGCTTCTTCTTCGCCTTCACCTTCTACCAC...TTC'), id='AGX93051.1', name='AGX93051.1', description='AGX93051.1 geraniol 10-hydroxylase-like protein [Cinchona calisaya]', dbxrefs=[]),
 SeqRecord(seq=Seq('ATGCGGATGTT

In [69]:
aa_seq_codon = codon_optimized_seqs[0].translate()

In [70]:
from Bio.Seq import Seq

In [71]:
first = Seq(str(fasta_seqs[0].seq))
aa_seq_original = str(first.translate())
aa_seq_original

'MDYLTIILTLLFALTLYEAFSYLSRRTKNLPPGPSPLPFIGSLHLLGDQPHKSLAKLSKKHGPIMSLKLGQITTIVISSSTMAKEVLQKQDLAFSSRSVPNALHAHNQFKFSVVWLPVASRWRSLRKVLNSNIFSGNRLDANQHLRTRKVQELIAYCRKNSQSGEAVDVGRAAFRTSLNLLSNLIFSKDLTDPYSDSAKEFKDLVWNIMVEAGKPNLVDFFPLLEKVDPQGIRHRMTIHFGEVLKLFGGLVNERLEQRRSKGEKNDVLDVLLTTSQESPEEIDRTHIERMCLDLFVAGTDTTSSTLEWAMSEMLKNPDKMKKTQDELAQVIGRGKTIEESDINRLPYLRCVMKETLRIHPPVPFLIPRKVEQSVEVCGYNVPKGSQVLVNAWAIGRDETVWDDALAFKPERFMESELDIRGRDFELIPFGAGRRICPGLPLALRTVPLMLGSLLNSFNWKLEGGMAPKDLDMEEKFGITLQKAHPLRAVPSTL'

In [72]:
print(aa_seq_original)
print(aa_seq_codon.seq)

MDYLTIILTLLFALTLYEAFSYLSRRTKNLPPGPSPLPFIGSLHLLGDQPHKSLAKLSKKHGPIMSLKLGQITTIVISSSTMAKEVLQKQDLAFSSRSVPNALHAHNQFKFSVVWLPVASRWRSLRKVLNSNIFSGNRLDANQHLRTRKVQELIAYCRKNSQSGEAVDVGRAAFRTSLNLLSNLIFSKDLTDPYSDSAKEFKDLVWNIMVEAGKPNLVDFFPLLEKVDPQGIRHRMTIHFGEVLKLFGGLVNERLEQRRSKGEKNDVLDVLLTTSQESPEEIDRTHIERMCLDLFVAGTDTTSSTLEWAMSEMLKNPDKMKKTQDELAQVIGRGKTIEESDINRLPYLRCVMKETLRIHPPVPFLIPRKVEQSVEVCGYNVPKGSQVLVNAWAIGRDETVWDDALAFKPERFMESELDIRGRDFELIPFGAGRRICPGLPLALRTVPLMLGSLLNSFNWKLEGGMAPKDLDMEEKFGITLQKAHPLRAVPSTL
MEYFTMMFTFFFAFTFYDAFSYFSRRTNKFPPGPSPFPFMGSFQFFGEHPQNSFANFSNNQGPMMSFNFGHMTTMVMSSSTMANDVFHNHEFAFSSRSVPKAFQAQKHFNFSVVWFPVASRWRSFRNVFKSKMFSGKRFEAKHQFRTRNVHDFMAYWRNKSHSGDAVEVGRAAFRTSFKFFSKFMFSNEFTEPYSESANDFNEFVWKMMVDAGNPKFVEFFPFFDNVEPHGMRQRMTMQFGDVFNFFGGFVKDRFDHRRSNGDNKEVFEVFFTTSHDSPDDMERTQMDRMWFEFFVAGTETTSSTFDWAMSDMFNKPENMNNTHEDFAHVMGRGNTMDDSEMKRFPYFRWVMNDTFRMQPPVPFFMPRNVDHSVDVWGYKVPNGSHVFVKAWAMGREDTVWEEAFAFNPDRFMDSDFEMRGREFDFMPFGAGRRMWPGFPFAFRTVPFMFGSFFKSFKWNFDGGMAPNEFEMDDNFGMTFHNAQPFRAVPSTF


In [73]:
str(aa_seq_codon.seq)

'MEYFTMMFTFFFAFTFYDAFSYFSRRTNKFPPGPSPFPFMGSFQFFGEHPQNSFANFSNNQGPMMSFNFGHMTTMVMSSSTMANDVFHNHEFAFSSRSVPKAFQAQKHFNFSVVWFPVASRWRSFRNVFKSKMFSGKRFEAKHQFRTRNVHDFMAYWRNKSHSGDAVEVGRAAFRTSFKFFSKFMFSNEFTEPYSESANDFNEFVWKMMVDAGNPKFVEFFPFFDNVEPHGMRQRMTMQFGDVFNFFGGFVKDRFDHRRSNGDNKEVFEVFFTTSHDSPDDMERTQMDRMWFEFFVAGTETTSSTFDWAMSDMFNKPENMNNTHEDFAHVMGRGNTMDDSEMKRFPYFRWVMNDTFRMQPPVPFFMPRNVDHSVDVWGYKVPNGSHVFVKAWAMGREDTVWEEAFAFNPDRFMDSDFEMRGREFDFMPFGAGRRMWPGFPFAFRTVPFMFGSFFKSFKWNFDGGMAPNEFEMDDNFGMTFHNAQPFRAVPSTF'

In [74]:
str(aa_seq_original) ==  str(aa_seq_codon.seq)

False

In [58]:
SeqRecord(str(fasta_seqs[0].seq)).translate()

TypeError: translate() takes no keyword arguments

In [92]:
my_seq = 'ATGTTTGGGAAA'

In [91]:
Seq(my_seq).translate()

Seq('MFGK')

In [100]:
from dnachisel import *

# DEFINE THE OPTIMIZATION PROBLEM

problem = DnaOptimizationProblem(
    sequence=my_seq, objectives=[CodonOptimize(codon_usage_table=aspergillus_oryzae, method='use_best_codon')])

# SOLVE THE CONSTRAINTS, OPTIMIZE WITH RESPECT TO THE OBJECTIVE

problem.resolve_constraints()
problem.optimize()

# PRINT SUMMARIES TO CHECK THAT CONSTRAINTS PASS

#print(problem.constraints_text_summary())
print(problem.objectives_text_summary())

# GET THE FINAL SEQUENCE (AS STRING OR ANNOTATED BIOPYTHON RECORDS)

final_sequence = problem.sequence  # string

location:   0%|                                  | 0/3 [00:00<?, ?it/s, now=3-6]
                                                                                

===> TOTAL OBJECTIVES SCORE:         0
✔        0 ┍ MaximizeCAI[0-12]((custom table)) 
           │ Codon opt. on window 0-12 scored -0.00E+00




In [101]:
final_sequence

'ATGTTCGGCAAC'

In [102]:
Seq(final_sequence).translate()

Seq('MFGN')

In [93]:
import python_codon_tables as pct

# PRINT THE LIST OF NAMES OF ALL AVAILABLE TABLES
print("Available tables:", pct.available_codon_tables_names)

# LOAD ONE TABLE BY NAME
table = pct.get_codons_table("b_subtilis_1423")
print(table["T"]["ACA"])  # returns 0.4
print(table["*"]["TAA"])  # returns 0.61


# LOAD ALL TABLES AT ONCE
codon_tables = pct.get_all_available_codons_tables()
print(codon_tables["c_elegans_6239"]["L"]["CTA"])  # returns 0.09

Available tables: ['b_subtilis_1423', 'd_melanogaster_7227', 'm_musculus_domesticus_10092', 'm_musculus_10090', 'e_coli_316407', 'g_gallus_9031', 'c_elegans_6239', 's_cerevisiae_4932', 'h_sapiens_9606']
0.4
0.61
0.09


In [94]:
from python_codon_tables import get_codons_table

In [99]:
from python_codon_tables import get_codons_table
aspergillus_oryzae = get_codons_table(5062) # aspergillus taxid = 5062
aspergillus_oryzae

{'*': {'TAA': 0.33, 'TAG': 0.29, 'TGA': 0.38},
 'A': {'GCA': 0.23, 'GCC': 0.3, 'GCG': 0.2, 'GCT': 0.27},
 'C': {'TGC': 0.54, 'TGT': 0.46},
 'D': {'GAC': 0.47, 'GAT': 0.53},
 'E': {'GAA': 0.44, 'GAG': 0.56},
 'F': {'TTC': 0.62, 'TTT': 0.38},
 'G': {'GGA': 0.24, 'GGC': 0.31, 'GGG': 0.17, 'GGT': 0.28},
 'H': {'CAC': 0.47, 'CAT': 0.53},
 'I': {'ATA': 0.14, 'ATC': 0.5, 'ATT': 0.36},
 'K': {'AAA': 0.36, 'AAG': 0.64},
 'L': {'CTA': 0.11,
  'CTC': 0.23,
  'CTG': 0.22,
  'CTT': 0.19,
  'TTA': 0.07,
  'TTG': 0.18},
 'M': {'ATG': 1.0},
 'N': {'AAC': 0.55, 'AAT': 0.45},
 'P': {'CCA': 0.25, 'CCC': 0.26, 'CCG': 0.22, 'CCT': 0.27},
 'Q': {'CAA': 0.43, 'CAG': 0.57},
 'R': {'AGA': 0.13,
  'AGG': 0.12,
  'CGA': 0.17,
  'CGC': 0.23,
  'CGG': 0.18,
  'CGT': 0.18},
 'S': {'AGC': 0.18,
  'AGT': 0.13,
  'TCA': 0.14,
  'TCC': 0.2,
  'TCG': 0.16,
  'TCT': 0.18},
 'T': {'ACA': 0.23, 'ACC': 0.32, 'ACG': 0.2, 'ACT': 0.24},
 'V': {'GTA': 0.13, 'GTC': 0.33, 'GTG': 0.27, 'GTT': 0.27},
 'W': {'TGG': 1.0},
 'Y': {'TAC

In [3]:
import numpy as np
import pandas as pd

def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df
def generate_artificial_peptide(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, max_length=22) -> str:
    """
    Generate an artificial peptide based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
        
    Returns:
    -------
    str
        Generated artificial peptide
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    out_str = ''
    for i in range(len(list_of_probabilities)):
        # make synthetic signal peptide
        artificial_amino_acid = list(np.random.choice(amino_acids, 1, p=list_of_probabilities[i]))

        if artificial_amino_acid == ['-']: 
            break

        out_str += artificial_amino_acid[0]
    return out_str

def add_dunder_tail(peptide:str , max_lenght:int=22 ): 
    '''Adds a tail if a peptide is shorter than the specified max_len.
    '''
    if len(peptide) < max_lenght: 
        difference = max_lenght - len(peptide)
        sequence = peptide + ('-'*difference)
    else: 
        sequence = peptide
        
    return sequence      

def generate_artificial_peptides(list_of_probabilities: np.ndarray, amino_acids: np.ndarray, n_peptides: int, max_len=50) -> pd.DataFrame:
    """
    Generate a dataframe of artificial peptides based on a list of probabilities and amino acids.
    
    Parameters:
    ----------
    list_of_probabilities : numpy.ndarray
        2-D array of probability of amino acids in the peptide
    amino_acids : numpy.ndarray
        1-D array of amino acids.
    n_peptides : int
        Number of peptides to generate
        
    Returns:
    -------
    pd.DataFrame
        Dataframe of generated artificial peptides with 'sequence' as column
        
    Notes:
    ------
    The length of the probability array should be same as the length of the peptide.
    """
    artificial_peptides = []
    lengths = [] 
    for i in range(n_peptides): 
        peptide = generate_artificial_peptide(list_of_probabilities,amino_acids, max_length=max_len)
        if len(peptide) <= max_len:
            peptide_w_tail = add_dunder_tail(peptide, max_lenght = max_len)
        else: 
            continue
        
        # save
        lengths.append(len(peptide))                                     
        artificial_peptides.append(peptide_w_tail)

    df = pd.DataFrame(artificial_peptides, columns =['sequence'])
    df['length'] = lengths
    return df



df_pwn = pd.read_csv('../data/02_all_signal_peptides/df_pwn_for_signal_peptides_found_in_supernatant.csv', index_col = False)
df_pwn
amino_acids = list(df_pwn.columns.values)
amino_acids
list_of_probabilities = []
for i in range(len(df_pwn)): 
    list_of_probabilities.append(df_pwn.loc[i, :].values.tolist())
list_of_probabilities[1]
len(list_of_probabilities)
list_of_probabilities_random = list_of_probabilities.copy()
# GEnerating random signal peptides with equal distribution
for i in range(1,len(list_of_probabilities_random)): 
    for j in range(len(list_of_probabilities_random[i])):
        list_of_probabilities_random[i][j] = (100/21)/100
        



random_peptides = generate_artificial_peptides(list_of_probabilities_random,amino_acids, 
                                                n_peptides=50000, 
                                                max_len=22)


random_peptides_15_22 = random_peptides[random_peptides["length"] >= 15]

random_peptides_15_22.to_csv('../data/05_best_signal_peptides/random_peptides_for_ML_training/random_peptides_start_M_for_ML_training_15_22.csv')
random_peptides_15_22


,sequence,length
1,MTQFQKHMNFDKPFNGARRKP-,21
8,MAISSMVYGQAKHISFLYF---,19
10,MHALWCGKFHNVEEC-------,15
11,MTDIAKQRNEIHNLSAT-----,17
17,MFFGKQDQVEYPYHIAGVRRPL,22
...,...,...
32970,MCLIYKPVSNNLQQWAI-----,17
32972,MEFPYDPVHADNECR-------,15
32987,MDAVFYFMYCNGTENTDIT---,19
32991,MKIMHQKITFYAPILEAAAKW-,21


In [16]:
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [23]:
list_of_signal_peptides = random_peptides_15_22['sequence'].to_list()

list_of_signal_peptides_pure_seqs  =[]

for peptide in list_of_signal_peptides: 
    pure_peptide = peptide.replace('-', '')
    list_of_signal_peptides_pure_seqs.append(pure_peptide)

list_of_signal_peptides_pure_seqs

['MTQFQKHMNFDKPFNGARRKP',
 'MAISSMVYGQAKHISFLYF',
 'MHALWCGKFHNVEEC',
 'MTDIAKQRNEIHNLSAT',
 'MFFGKQDQVEYPYHIAGVRRPL',
 'MQDMWWNCNMGKCVRWVV',
 'MTQLPCYHHRKAQEGRTCYC',
 'MKMFNSPHAWYAHTQEVDSHE',
 'MEMGQRRKIWNKDRA',
 'MHRWEVAFMTTMDEEL',
 'MDFAKFNHSQNDTPR',
 'MTSHTTGTVPISRPICD',
 'MWVQLEHACFDKTPLRA',
 'MKMHYVLCWQHVCFMHTYQMR',
 'MNACHRMFPRGWFLKSMR',
 'MFCLMNMLYELAINDK',
 'MEIFRGMCRDYAAEAHH',
 'MKFNMSFFYANHARTCPRHDYY',
 'MRNEIVENRPSCCNC',
 'MYRSMSDHLYQWYTWNRYW',
 'MMAPQMKVCYPKHGHMAPTS',
 'MKGGQLRQHDECRMLCMSH',
 'MMPNPSYSWDQCCKTP',
 'MHWAYKRDMLMRYYL',
 'MHMRTVVVCCPSEDEV',
 'MYNNRWNHSMIGKKHKDI',
 'MHVDPFRFRPFQDINWIEVYCT',
 'MDFLARDRNAEAWDGLPCMR',
 'MEVTQQMISIVYYEQVNCYVSA',
 'MMESLHIIARRHLKI',
 'MLLMNDVGWLFQHPL',
 'MCTCEMSWKRHVFEWDGNP',
 'MWRSWDMRKHGHCWGHAAVHQT',
 'MWVACQCGYIFPEQLQ',
 'MYTESIKHWQHNNESQHFSPDE',
 'MYMKLGSCELKMINVA',
 'MHNSKCINHCMKFMRQH',
 'MAKVQTHIGYMRCVP',
 'MILWVTQEAMAIFVDHHQPPLQ',
 'MCIHLNQHHLRHTMWAG',
 'MLDNHFCWHFCWISEHNWAK',
 'MHNEKAEADSNLQPGMRW',
 'MWFPQDHSPPNRSMWL',
 'MQYP

In [24]:
list_of_signal_peptides_pure_seqs_seqRecods = [SeqRecord(Seq(peptide)) for peptide in list_of_signal_peptides_pure_seqs]

In [27]:
# add gfp
GFP = 'MSKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTFSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'
list_of_signal_peptides_pure_seqs_seqRecods = [SeqRecord(Seq(peptide+GFP))  for peptide in list_of_signal_peptides_pure_seqs]


In [31]:
list_of_signal_peptides_pure_seqs_seqRecods = list_of_signal_peptides_pure_seqs_seqRecods[:5000]

In [35]:
for i in range(len(list_of_signal_peptides_pure_seqs_seqRecods)):
    list_of_signal_peptides_pure_seqs_seqRecods[i].id = str(i+1)
    list_of_signal_peptides_pure_seqs_seqRecods[i].name = f'sp_random_{i+1:00}'
    list_of_signal_peptides_pure_seqs_seqRecods[i].description = f'sp_random_{i+1:00}'

list_of_signal_peptides_pure_seqs_seqRecods

In [36]:
from Bio import SeqIO

with open(f"../data/05_best_signal_peptides/random_peptides_for_ML_training/random_signal_peptides_with_AA_dist_equal.fasta", "w") as output_handle:
    SeqIO.write(list_of_signal_peptides_pure_seqs_seqRecods, output_handle, "fasta")

In [45]:
random_signal_peptides_predicted_bySignalP = pd.read_csv('../data/05_best_signal_peptides/random_peptides_for_ML_training/prediction_results.txt',sep='\t', header=(1), index_col=False)
random_signal_peptides_predicted_bySignalP

,# ID,Prediction,OTHER,SP(Sec/SPI),CS Position
0,1 sp_random_1,OTHER,1.000009,0.0,NaN
1,2 sp_random_2,OTHER,1.000011,0.0,NaN
2,3 sp_random_3,OTHER,1.000016,0.0,NaN
3,4 sp_random_4,OTHER,1.000002,0.0,NaN
4,5 sp_random_5,OTHER,1.000031,0.0,NaN
...,...,...,...,...,...
4995,4996 sp_random_4996,OTHER,0.999977,0.0,NaN
4996,4997 sp_random_4997,OTHER,1.000014,0.0,NaN
4997,4998 sp_random_4998,OTHER,1.000000,0.0,NaN
4998,4999 sp_random_4999,OTHER,1.000013,0.0,NaN


In [53]:
df_signalP_filtered = random_signal_peptides_predicted_bySignalP[random_signal_peptides_predicted_bySignalP["OTHER"] >= 0.99]
df_signalP_filtered

,# ID,Prediction,OTHER,SP(Sec/SPI),CS Position
0,1 sp_random_1,OTHER,1.000009,0.0,NaN
1,2 sp_random_2,OTHER,1.000011,0.0,NaN
2,3 sp_random_3,OTHER,1.000016,0.0,NaN
3,4 sp_random_4,OTHER,1.000002,0.0,NaN
4,5 sp_random_5,OTHER,1.000031,0.0,NaN
...,...,...,...,...,...
4995,4996 sp_random_4996,OTHER,0.999977,0.0,NaN
4996,4997 sp_random_4997,OTHER,1.000014,0.0,NaN
4997,4998 sp_random_4998,OTHER,1.000000,0.0,NaN
4998,4999 sp_random_4999,OTHER,1.000013,0.0,NaN


In [54]:
df_signalP_filtered = df_signalP_filtered['# ID'].to_list()
filtered_names = [name.split(" ")[1] for name in df_signalP_filtered]
filtered_names

['sp_random_1',
 'sp_random_2',
 'sp_random_3',
 'sp_random_4',
 'sp_random_5',
 'sp_random_6',
 'sp_random_7',
 'sp_random_8',
 'sp_random_9',
 'sp_random_10',
 'sp_random_11',
 'sp_random_12',
 'sp_random_13',
 'sp_random_14',
 'sp_random_15',
 'sp_random_16',
 'sp_random_17',
 'sp_random_18',
 'sp_random_19',
 'sp_random_20',
 'sp_random_21',
 'sp_random_22',
 'sp_random_23',
 'sp_random_24',
 'sp_random_25',
 'sp_random_26',
 'sp_random_27',
 'sp_random_28',
 'sp_random_29',
 'sp_random_30',
 'sp_random_31',
 'sp_random_32',
 'sp_random_33',
 'sp_random_34',
 'sp_random_35',
 'sp_random_36',
 'sp_random_37',
 'sp_random_38',
 'sp_random_39',
 'sp_random_40',
 'sp_random_41',
 'sp_random_42',
 'sp_random_43',
 'sp_random_44',
 'sp_random_45',
 'sp_random_46',
 'sp_random_47',
 'sp_random_48',
 'sp_random_49',
 'sp_random_50',
 'sp_random_51',
 'sp_random_52',
 'sp_random_53',
 'sp_random_54',
 'sp_random_55',
 'sp_random_56',
 'sp_random_57',
 'sp_random_58',
 'sp_random_59',
 'sp_r

In [56]:
not_a_signal_peptide = []
for sp in list_of_signal_peptides_pure_seqs_seqRecods: 
    if sp.name in filtered_names: 
        not_a_signal_peptide.append(sp)

In [59]:
len(not_a_signal_peptide)

4852

In [60]:
with open(f"../data/05_best_signal_peptides/random_peptides_for_ML_training/random_signal_peptides_with_AA_dist_equal_no_SP_by_SIGNALP.fasta", "w") as output_handle:
    SeqIO.write(not_a_signal_peptide, output_handle, "fasta")

In [63]:
sp22 = [peptide[:22] for peptide in not_a_signal_peptide]

In [73]:
sequences = [str(peptide[:22].seq) for peptide in not_a_signal_peptide]
names = [peptide.name for peptide in not_a_signal_peptide]
prediction = [0.0 for i in range(len(not_a_signal_peptide))]
df_22_no_sp = pd.DataFrame({'name':names, 'sequence':sequences, 'abundance':prediction})
df_22_no_sp.to_csv('../data/05_best_signal_peptides/random_peptides_for_ML_training/no_sp_predicted_by_signalP_22.csv', index=False)
df_22_no_sp

,name,sequence,abundance
0,sp_random_1,MTQFQKHMNFDKPFNGARRKPM,0.0
1,sp_random_2,MAISSMVYGQAKHISFLYFMSK,0.0
2,sp_random_3,MHALWCGKFHNVEECMSKGEEL,0.0
3,sp_random_4,MTDIAKQRNEIHNLSATMSKGE,0.0
4,sp_random_5,MFFGKQDQVEYPYHIAGVRRPL,0.0
...,...,...,...
4847,sp_random_4996,MDNNFYQGHTKSDFKMRWLNMS,0.0
4848,sp_random_4997,MTVEQLNNCCGIPSKQMSKGEE,0.0
4849,sp_random_4998,MHAMELQGEEPEAHLCGRIYFM,0.0
4850,sp_random_4999,MFGYNAQSPYENRVTQFMSKGE,0.0


In [12]:
#Lets run these through deeploc. 
from Bio import SeqIO
import pandas as pd

path_to_file = '../data/00_All_proteins_and_partitions_Aoryzae_and_/FungiDB-59_AoryzaeRIB40_AnnotatedProteins.fasta'

# Open file with "with" statement to avoid problems with access 
proteins = []
with open(path_to_file, mode='r') as handle:
    for record in SeqIO.parse(handle, 'fasta'):
        proteins.append(record)

# signal_peptides
df_signalPP = pd.read_csv('../data/03_proteomics_data/ML_signal_peptides.csv')
list_of_protemics_acc_numbers = df_signalPP['Accession'].to_list() # here put the list

#Cross ref
new_list_of_the_proteins_for_deeploc = []
for seq in proteins:
    if seq.id in list_of_protemics_acc_numbers:
        new_list_of_the_proteins_for_deeploc.append(seq)

In [13]:
partition1 = new_list_of_the_proteins_for_deeploc[:500]
partition2 = new_list_of_the_proteins_for_deeploc[500:1000]
partition3 = new_list_of_the_proteins_for_deeploc[1000:]

# write it to a fasta file
with open(f"../data/00_All_proteins_and_partitions_Aoryzae_and_/all_sps_to_deeploc_proteins1.fasta", "w") as output_handle:
    SeqIO.write(partition1, output_handle, "fasta")

with open(f"../data/00_All_proteins_and_partitions_Aoryzae_and_/all_sps_to_deeploc_proteins2.fasta", "w") as output_handle:
    SeqIO.write(partition2, output_handle, "fasta")


with open(f"../data/00_All_proteins_and_partitions_Aoryzae_and_/all_sps_to_deeploc_proteins3.fasta", "w") as output_handle:
    SeqIO.write(partition3, output_handle, "fasta")

In [24]:
import time
from datetime import date
today = date.today().isoformat()
today = date.today()

today

datetime.date(2023, 5, 19)

In [30]:
from datetime import datetime, date, time, timezone
dt = datetime.strptime("21/11/06 16:30", "%d/%m/%y %H:%M").isoformat()
dt


'2006-11-21T16:30:00'

In [32]:
datetime.now().isoformat()   

'2023-05-19T14:44:40.779456'